In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [53]:
def sound_speed(P, rho):
    """Calculate sound speed."""
    epsilon = 1e-6  # Small value to avoid division by zero
    P = abs(P)  # Ensure pressure is non-negative
    
    return np.sqrt(gamma * P / rho+epsilon )

In [54]:
def compute_flux(U):
    """Compute the flux vector F for a single row of conserved variables."""
    rho, rho_v, E = U  # Extract variables

    # Ensure non-zero density and energy
    rho = max(rho, 1e-8)  # Avoid zero density
    E = max(E, 1e-8)      # Avoid zero or negative energy

    # Compute velocity and pressure
    v = rho_v / rho  # Velocity
    P = max((gamma - 1) * (E - 0.5 * rho * v ** 2), 0.0)  # Pressure

    # Compute fluxes
    F = np.zeros(3)
    F[0] = rho_v  # Mass flux
    F[1] = rho_v * v + P  # Momentum flux
    F[2] = (E + P) * v  # Energy flux

    return F


In [55]:
def HLL_flux(UL, UR):
    """HLL Riemann solver to compute flux at the interface."""
    FL = compute_flux(UL)
    FR = compute_flux(UR)

    # Extract variables from left and right states
    rhoL, vL = UL[0], UL[1] / max(UL[0], 1e-8)
    rhoR, vR = UR[0], UR[1] / max(UR[0], 1e-8)

    # Compute pressures
    PL = max((gamma - 1) * (UL[2] - 0.5 * rhoL * vL ** 2), 0.0)
    PR = max((gamma - 1) * (UR[2] - 0.5 * rhoR * vR ** 2), 0.0)

    # Compute sound speeds
    cL = sound_speed(PL, rhoL)
    cR = sound_speed(PR, rhoR)

    # Compute wave speeds
    alpha_minus = min(vL - cL, vR - cR)  # Leftward moving wave speed
    alpha_plus = max(vL + cL, vR + cR)   # Rightward moving wave speed

    # Compute the maximum wave speed for CFL condition
    alpha_max = max(abs(alpha_minus), abs(alpha_plus))

    # Compute the HLL flux based on wave speeds
    if alpha_minus >= 0:
        flux = FL  # Use left flux if all waves move to the right
    elif alpha_plus <= 0:
        flux = FR  # Use right flux if all waves move to the left
    else:
        # Intermediate state: mix fluxes
        flux = (alpha_plus * FL + alpha_minus * FR -
                alpha_plus * alpha_minus * (UR - UL)) / (alpha_plus - alpha_minus)

    return flux, alpha_max  # Return both flux and max wave speed



In [56]:
def check_for_nan(U):
    """Check for NaN values and stop the simulation if any are found."""
    if np.any(np.isnan(U)):
        print("NaN detected! Stopping simulation.")
        print(U)
        raise ValueError("NaN values encountered!")

In [65]:
def update(U, dt):
    """Update conserved variables using fluxes."""
    U_new = np.copy(U)
    global_max_alpha = 1e-8  # Track the maximum wave speed across the entire grid

    for i in range(1, len(U) - 1):
        # Compute fluxes and wave speeds at the interfaces
        flux_left, alpha_left = HLL_flux(U[i - 1], U[i])
        flux_right, alpha_right = HLL_flux(U[i], U[i + 1])

        # Track the maximum wave speed for CFL condition
        global_max_alpha = max(global_max_alpha, alpha_left, alpha_right)

        # Update the conserved variables
        print(flux_right - flux_left)
        U_new[i] = U[i] - (0.5 * global_max_alpha) * (flux_right - flux_left) # 0.5 * global_max_alpha = dt/dx (0.5 is cfl)

    return U_new, global_max_alpha  # Return updated state and max alpha


In [66]:
# Parameters
gamma = 1.4  # Adiabatic index for ideal gas
Nx = 5  # Number of grid cells
x_min, x_max = 0.0, 1.0  # Domain
dx = (x_max - x_min) / Nx  # Grid spacing

# Time step parameters
CFL = 0.5  # Courant number for stability
t_max = 0.5  # Maximum simulation time

In [67]:
# Initialize arrays for conserved variables: U[rho, rho*v, E]
U = np.zeros((Nx, 3))  # U[i] = [rho_i, rho*v_i, E_i]

# Initial conditions for a shock tube
U[: len(U) // 2, 0] = 1.0  # Left half: density (rho) = 1.0
U[len(U) // 2 :, 0] = 0.125  # Right half: density (rho) = 0.125

U[: len(U) // 2, 2] = 2.5  # Left half: energy = 2.5
U[len(U) // 2 :, 2] = 0.25  # Right half: energy = 0.25



In [68]:
# Time integration loop
t = 0.0
dt = 0

while t < t_max:
    # Update the state and compute the global maximum wave speed
    U, max_alpha = update(U, dt)

    # Check for NaN values
    check_for_nan(U)

    # Compute the new time step based on CFL condition
    dt = CFL * dx / max_alpha

    # Increment the time
    t += dt

print(U)


[-0.51765717  0.45       -1.33111843]
[ 0.51765717 -0.45        1.33111843]
[0. 0. 0.]
[-1.45850862e+00 -4.15464990e+06 -3.74471635e+00]
[nan nan nan]
[nan nan nan]
NaN detected! Stopping simulation.
[[1.00000000e+00 0.00000000e+00 2.50000000e+00]
 [2.31704899e+00 2.87932108e+06 5.88272326e+00]
 [           nan            nan            nan]
 [           nan            nan            nan]
 [1.25000000e-01 0.00000000e+00 2.50000000e-01]]


<ipython-input-53-e3fa8ea35d7e>:6: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(gamma * P / rho+epsilon )


ValueError: NaN values encountered!